# Model Evaluation

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import os
import sys
sys.path.append("..")
from cache import cache

In [2]:
IMAGE_DIR = r"D:\data\train-256\train"
DATA_OUTPUT_DIR = r"D:\data\VRNet"

In [3]:
DEPTH = 9

In [4]:
#TODO Move all the relatoinship manipulation into its own file

In [5]:
IMAGE_DIR_REL = r"D:\data\VRNet\imagenet_rel_subset"

In [6]:
all_relationships_file = r"D:\data\open-images-2019-visual-relationship\challenge-2019-train-vrd.csv"
all_relationships = pd.read_csv(all_relationships_file)

In [7]:
rel_definition_file = r"D:\data\open-images-2019-visual-relationship\challenge-2019-relationship-triplets.csv"
rel_definition = pd.read_csv(rel_definition_file)

In [8]:
rel_counts = all_relationships.ImageID.value_counts()

In [9]:
# Do not use 'is' relationships
all_relationships_non_is = all_relationships[all_relationships.RelationshipLabel != "is"]
len(all_relationships_non_is)

180626

In [10]:
images_w_relationships = all_relationships_non_is.ImageID.unique()
len(images_w_relationships)

57187

In [11]:
images_w_relationships_paths = [os.path.join(IMAGE_DIR_REL, (f + '.jpg')) for f in images_w_relationships]
len(images_w_relationships_paths)

57187

In [12]:
imageIDs_trainz = images_w_relationships

In [13]:
filenames_trainz = [os.path.join(IMAGE_DIR_REL, (f + '.jpg')) for f in imageIDs_trainz]
len(filenames_trainz)

57187

In [14]:
# Get training relationships
trainz_relationships = all_relationships_non_is[all_relationships_non_is.ImageID.isin(imageIDs_trainz)]
len(trainz_relationships)

180626

### class dictionary

In [15]:
import csv
class_dict = {}
reader = csv.reader(open(r"D:\data\open-images-2019-visual-relationship\challenge-2019-classes-vrd.csv"))
for row in reader:
    k, v = row
    class_dict[k] = v.replace(' ', '').replace('(', '').replace(')', '').lower()

reader = csv.reader(open( r"D:\data\open-images-2019-visual-relationship\challenge-2019-attributes-description.csv"))
for row in reader:
    k, v = row
    class_dict[k] = v.replace(' ', '').replace('(', '').replace(')', '').lower()

In [16]:
trainz_relationships['LabelClass1'] = trainz_relationships.LabelName1.apply(lambda x: class_dict[x])
trainz_relationships['LabelClass2'] = trainz_relationships.LabelName2.apply(lambda x: class_dict[x])

***

In [17]:
def get_relationship_triplet_list():

    trainz_relationship_list = []
    i = 0
    for iid in imageIDs_trainz:
        print_progress(i, len(imageIDs_trainz))
        trainz_relationship_list.append(trainz_relationships.loc[trainz_relationships.ImageID == iid, \
                                                                 ["LabelClass1", "LabelClass2", "RelationshipLabel"]].values)
        i = i+1
    return trainz_relationship_list

In [18]:
%%time
cache_path = os.path.join(DATA_OUTPUT_DIR,
                              "trainz_relationship_list.pkl")

trainz_relationship_list = cache(cache_path=cache_path,
                        fn=get_relationship_triplet_list)

len(trainz_relationship_list)

- Data loaded from cache-file: D:\data\VRNet\trainz_relationship_list.pkl
Wall time: 234 ms


57187

In [19]:
# Split train and validation
testz_relationship_list = trainz_relationship_list[:5700]

trainz_relationship_list = trainz_relationship_list[:-5700]

len(trainz_relationship_list), len(testz_relationship_list)

(51487, 5700)

In [20]:
trainz_relationship_list_joined = []
for relations in trainz_relationship_list:
    trainz_relationship_list_joined.append(list(set([' '.join(rel) for rel in relations])))

In [21]:
testz_relationship_list_joined = []
for relations in trainz_relationship_list:
    testz_relationship_list_joined.append(list(set([' '.join(rel) for rel in relations])))

In [22]:
def flatten(relations_listlist):
    relations_list = [relation
                     for relations_list in relations_listlist
                     for relation in relations_list]
    
    return relations_list

In [23]:
%%time
from glob import glob
import pickle

predictions = [(pickle.load(open(f, 'rb')), f) for f in 
              glob(os.path.join(DATA_OUTPUT_DIR, 
                                f"trainz_*_*_predictions.pkl"))]
###########################################
predictions_c = predictions
for i, (prediction, filename) in enumerate(predictions):
    for j, pred in enumerate(prediction):
        if 'inside of' in pred or 'interacts with' in pred:
            pred = pred.replace('inside of', 'inside_of').replace('interacts with', 'interacts_with')
            predictions_c[i][0][j] = pred
###########################################
for prediction, filename in predictions_c:
    # train
    precision = [predicted in actual 
             for predicted, actual in zip(prediction, trainz_relationship_list_joined)]
    # test
    t_precision = [predicted in actual 
             for predicted, actual in zip(prediction[:5700], testz_relationship_list_joined)]
    
    name = os.path.basename(filename).replace('trainz_', '').replace('_predictions.pkl', '')
#     print(f"test:\t {np.sum(t_precision)}/{len(precision)}\t {np.sum(t_precision)/len(t_precision)} : {name}")
#     print(f"train:\t {np.sum(precision)}/{len(precision)}\t {np.sum(precision)/len(precision)} : {name}")
    print(f"{name[:-2]}, {name[-1]}, t, {np.sum(precision)}, {len(precision)}, {len(flatten(trainz_relationship_list))}")
    print(f"{name[:-2]}, {name[-1]}, v, {np.sum(t_precision)}, {len(t_precision)}, {len(flatten(testz_relationship_list))}")

inception_v3, 3, t, 45609, 51487, 174106
inception_v3, 3, v, 4951, 5700, 39683
inception_v3, 6, t, 42190, 51487, 174106
inception_v3, 6, v, 4688, 5700, 39683
inception_v3, 9, t, 46471, 51487, 174106
inception_v3, 9, v, 5019, 5700, 39683
mobilenetv2_1.00_224, 3, t, 48270, 51487, 174106
mobilenetv2_1.00_224, 3, v, 5180, 5700, 39683
mobilenetv2_1.00_224, 6, t, 50352, 51487, 174106
mobilenetv2_1.00_224, 6, v, 5575, 5700, 39683
mobilenetv2_1.00_224, 9, t, 50218, 51487, 174106
mobilenetv2_1.00_224, 9, v, 5540, 5700, 39683
NASNet, 3, t, 47742, 51487, 174106
NASNet, 3, v, 5285, 5700, 39683
NASNet, 6, t, 49768, 51487, 174106
NASNet, 6, v, 5442, 5700, 39683
NASNet, 9, t, 50206, 51487, 174106
NASNet, 9, v, 5532, 5700, 39683
resnet101, 3, t, 21356, 51487, 174106
resnet101, 3, v, 2534, 5700, 39683
resnet101, 6, t, 18884, 51487, 174106
resnet101, 6, v, 2505, 5700, 39683
resnet101, 9, t, 21126, 51487, 174106
resnet101, 9, v, 2746, 5700, 39683
resnet50, 6, t, 18064, 51487, 174106
resnet50, 6, v, 2505,

***